In [1]:
import os
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from scipy import signal

In [10]:
pd.read_csv("/Data/CRS25/PPG_Certifiation/data/PPG_ECG_Data/user_1_final.csv").columns

Index(['Index', 'PPG', 'temperature', 'ECG'], dtype='object')

In [11]:
import os
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from scipy import signal

class MultiModalDataset(Dataset):
    def __init__(self, data_folder, window_size=300, stride=50, fs=128):
        self.window_size = window_size
        self.stride = stride
        self.fs = fs
        
        self.samples_ppg = []
        self.samples_temp = []
        self.labels = []
        
        # 1. 파일 리스트 로드
        search_pattern = os.path.join(data_folder, "user_*.csv")
        file_list = glob.glob(search_pattern)
        
        if not file_list:
            print(f"❌ 경고: '{data_folder}' 경로에서 파일을 하나도 찾지 못했습니다.")
            return

        print(f"📂 데이터 로딩 시작... (총 {len(file_list)}개 파일 감지)")
        
        for filepath in file_list:
            filename = os.path.basename(filepath)
            
            # 2. User ID 추출 (파일명 파싱)
            try:
                # user_4_part1_final.csv -> user, 4, part1, final.csv
                parts = filename.split('_')
                user_num = int(parts[1]) # 4 추출
                label = user_num - 1     # 0-based index
            except Exception as e:
                print(f"⚠️ 파일명 파싱 실패 ({filename}): {e}")
                continue

            # 3. CSV 읽기 및 전처리
            try:
                df = pd.read_csv(filepath)
                
                # [수정 완료] 확인된 컬럼명 직접 사용 ('Index', 'PPG', 'temperature')
                # 혹시 모를 공백 제거를 위해 컬럼명 strip 처리
                df.columns = [c.strip() for c in df.columns]
                
                if 'PPG' not in df.columns or 'temperature' not in df.columns:
                     print(f"  ❌ 컬럼 누락 ({filename}): {df.columns}")
                     continue

                raw_ppg = df['PPG'].values
                raw_temp = df['temperature'].values
                
                # 전처리: Detrending -> 4Hz Low-pass Filter
                processed_ppg = self.preprocess_ppg(raw_ppg)
                
                # 온도 정규화 (Min-Max Scaling: 25~40도 기준)
                processed_temp = (raw_temp - 25.0) / (40.0 - 25.0) 

                # 4. 슬라이딩 윈도우
                num_windows = (len(processed_ppg) - self.window_size) // self.stride
                
                if num_windows <= 0:
                    continue

                file_samples = 0
                for i in range(num_windows):
                    start = i * self.stride
                    end = start + self.window_size
                    
                    ppg_window = processed_ppg[start:end]
                    temp_window = processed_temp[start:end]
                    
                    # (Optional) 이상치 제거 (PPG Z-score > 5)
                    if np.max(np.abs(ppg_window)) > 5:
                        continue
                        
                    self.samples_ppg.append(ppg_window)
                    self.samples_temp.append(temp_window)
                    self.labels.append(label)
                    file_samples += 1
                
                print(f"  ✅ User {user_num} (Label {label}): {file_samples} windows loaded.")
                
            except Exception as e:
                print(f"  ❌ 데이터 로드 에러 ({filename}): {e}")

        # 리스트 -> Numpy -> Tensor
        self.samples_ppg = np.array(self.samples_ppg, dtype=np.float32)
        self.samples_temp = np.array(self.samples_temp, dtype=np.float32)
        self.labels = np.array(self.labels, dtype=np.int64)
        
        # 차원 확장: (N, 1, Length)
        if len(self.labels) > 0:
            self.samples_ppg = np.expand_dims(self.samples_ppg, axis=1)
            self.samples_temp = np.expand_dims(self.samples_temp, axis=1)

        print(f"🎉 전체 데이터 로드 완료! 총 샘플 수: {len(self.labels)}")

    def preprocess_ppg(self, signal_data):
        # 1. Detrending
        detrended = signal.detrend(signal_data)
        # 2. Low-pass Filter (4Hz)
        nyquist = 0.5 * self.fs
        cutoff = 4.0 / nyquist
        b, a = signal.butter(4, cutoff, btype='low')
        filtered = signal.filtfilt(b, a, detrended)
        # 3. Z-score Normalization
        normalized = (filtered - np.mean(filtered)) / (np.std(filtered) + 1e-6)
        return normalized

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return (torch.from_numpy(self.samples_ppg[idx]), 
                torch.from_numpy(self.samples_temp[idx])), torch.tensor(self.labels[idx])

In [12]:
if __name__ == "__main__":
    data_folder = "./data/PPG_ECG_Data"
    
    # 윈도우 크기 300 (논문 기준), 스트라이드 50 (겹쳐서 데이터 증강 효과)
    dataset = MultiModalDataset(data_folder, window_size=300, stride=50)
    
    # DataLoader 생성
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

    # 데이터 형상 확인
    if len(dataset) > 0:
        (ppg, temp), label = next(iter(dataloader))
        print("\n--- Batch Shape Check ---")
        print(f"PPG Input Shape : {ppg.shape}")   # (64, 1, 300)
        print(f"Temp Input Shape: {temp.shape}")  # (64, 1, 300)
        print(f"Label Shape     : {label.shape}") # (64,)

📂 데이터 로딩 시작... (총 18개 파일 감지)
  ✅ User 8 (Label 7): 135353 windows loaded.
  ✅ User 2 (Label 1): 113091 windows loaded.
  ✅ User 1 (Label 0): 113455 windows loaded.
  ✅ User 12 (Label 11): 116450 windows loaded.
  ✅ User 6 (Label 5): 86729 windows loaded.
  ✅ User 3 (Label 2): 122985 windows loaded.
  ✅ User 4 (Label 3): 75689 windows loaded.
  ✅ User 14 (Label 13): 133755 windows loaded.
  ✅ User 6 (Label 5): 31017 windows loaded.
  ✅ User 13 (Label 12): 132584 windows loaded.
  ✅ User 7 (Label 6): 121931 windows loaded.
  ✅ User 16 (Label 15): 127971 windows loaded.
  ✅ User 5 (Label 4): 122964 windows loaded.
  ✅ User 15 (Label 14): 122069 windows loaded.
  ✅ User 4 (Label 3): 56395 windows loaded.
  ✅ User 11 (Label 10): 121235 windows loaded.
  ✅ User 9 (Label 8): 133277 windows loaded.
  ✅ User 10 (Label 9): 114894 windows loaded.
🎉 전체 데이터 로드 완료! 총 샘플 수: 1981844

--- Batch Shape Check ---
PPG Input Shape : torch.Size([64, 1, 300])
Temp Input Shape: torch.Size([64, 1, 300])
Label S

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ==========================================
# 1. SSM Block (S4 Simplified)
# ==========================================
class S4Block(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super(S4Block, self).__init__()
        # 시계열의 시간적 특징(Long-range dependency)을 잡는 핵심 레이어
        # 논문의 복잡한 S4 수식을 1D Conv와 Gating 메커니즘으로 근사하여 구현
        self.conv1d = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, groups=d_model)
        self.norm = nn.LayerNorm(d_model)
        self.activation = nn.SiLU() # Mamba/S4에서 주로 쓰는 활성화 함수
        self.output_proj = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (Batch, Length, d_model)
        residual = x
        
        # 1. Time-mixing (Conv1D)
        # (B, L, D) -> (B, D, L)로 변환 후 Conv 적용
        x = x.transpose(1, 2)
        x = self.conv1d(x)
        x = x.transpose(1, 2)
        
        # 2. Gating
        x = self.activation(x)
        
        # 3. Channel-mixing (Linear)
        x = self.output_proj(x)
        x = self.dropout(x)
        
        # 4. Residual & Norm
        x = self.norm(x + residual)
        return x


In [14]:
# ==========================================
# 2. SSM Encoder
# ==========================================
class SSMEncoder(nn.Module):
    def __init__(self, input_len=300, d_model=64, num_layers=2):
        super(SSMEncoder, self).__init__()
        
        # 입력: (Batch, 1, 300) -> (Batch, 300, 64) 차원 확장
        self.input_projection = nn.Linear(1, d_model)
        
        # 논문대로 2개의 SSM 레이어 적층
        self.layers = nn.ModuleList([
            S4Block(d_model) for _ in range(num_layers)
        ])
        
        # 시퀀스를 하나의 벡터로 압축 (Pooling)
        self.pool = nn.AdaptiveAvgPool1d(1)

    def forward(self, x):
        # x: (Batch, 1, Length)
        
        # (B, 1, L) -> (B, L, 1) -> (B, L, d_model)
        x = x.transpose(1, 2)
        x = self.input_projection(x)
        
        # SSM Layers 통과
        for layer in self.layers:
            x = layer(x) # (B, L, d_model)
            
        # Global Pooling: (B, L, D) -> (B, D, 1) -> (B, D)
        x_pooled = self.pool(x.transpose(1, 2)).squeeze(-1)
        
        return x_pooled, x # Pooling된 벡터와 전체 시퀀스 모두 반환


In [15]:
# ==========================================
# 3. 전체 모델 (Fusion & Classifier)
# ==========================================
class CrossAttentionFusion(nn.Module):
    def __init__(self, num_users=16, d_model=64, num_heads=4):
        super(CrossAttentionFusion, self).__init__()
        
        # 듀얼 인코더 (PPG용, Temp용)
        self.ppg_encoder = SSMEncoder(d_model=d_model)
        self.temp_encoder = SSMEncoder(d_model=d_model)
        
        # Cross-Modal Attention
        self.cross_att = nn.MultiheadAttention(d_model, num_heads=num_heads, batch_first=True)
        
        self.norm_p = nn.LayerNorm(d_model)
        self.norm_t = nn.LayerNorm(d_model)
        
        # Fusion & Classifier
        self.fusion_fc = nn.Linear(d_model, d_model)
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(d_model // 2, num_users)
        )

    def forward(self, x_ppg, x_temp):
        # 1. 인코딩
        z_p, seq_p = self.ppg_encoder(x_ppg) # z: (B, D), seq: (B, L, D)
        z_t, seq_t = self.temp_encoder(x_temp)
        
        # 2. Cross-Attention을 위한 차원 조정 (B, 1, D)
        query_p = z_p.unsqueeze(1)
        query_t = z_t.unsqueeze(1)
        
        # PPG가 Temp를 참조하여 보정
        attn_out_p, _ = self.cross_att(query_p, query_t, query_t)
        z_p_refined = self.norm_p(z_p + attn_out_p.squeeze(1))
        
        # Temp가 PPG를 참조하여 보정
        attn_out_t, _ = self.cross_att(query_t, query_p, query_p)
        z_t_refined = self.norm_t(z_t + attn_out_t.squeeze(1))
        
        # 3. Fusion (Element-wise Addition)
        z_fused = z_p_refined + z_t_refined
        z_fused = F.relu(self.fusion_fc(z_fused))
        
        # 4. Classification
        logits = self.classifier(z_fused)
        
        return logits, z_p_refined, z_t_refined

In [16]:
class AlignmentLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super(AlignmentLoss, self).__init__()
        self.temperature = temperature
        self.ce_loss = nn.CrossEntropyLoss()

    def forward(self, z_a, z_b):
        # z_a, z_b: (Batch, D)
        # Cosine Similarity 계산
        z_a = F.normalize(z_a, dim=1)
        z_b = F.normalize(z_b, dim=1)
        
        # (B, D) @ (D, B) -> (B, B) 유사도 행렬
        logits = torch.matmul(z_a, z_b.T) / self.temperature
        
        # 정답: 대각선 요소 (자기 자신과의 쌍)
        labels = torch.arange(z_a.size(0)).to(z_a.device)
        
        return self.ce_loss(logits, labels)

class SpreadControlLoss(nn.Module):
    def __init__(self, threshold=0.001):
        super(SpreadControlLoss, self).__init__()
        self.threshold = threshold

    def forward(self, z):
        # 특징 벡터들의 분산이 너무 커지지 않도록 제어
        z = F.normalize(z, dim=1)
        var = torch.var(z, dim=0).mean()
        # 분산이 threshold보다 크면 페널티
        loss = F.relu(var - self.threshold)
        return loss

In [20]:
import torch.optim as optim
from tqdm.auto import tqdm  # Jupyter/Console 자동 감지

# ==========================================
# 설정 (Hyperparameters)
# ==========================================
NUM_USERS = 16  
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LAMBDA_A = 0.5  
LAMBDA_S = 0.01 
LR = 0.001
EPOCHS = 15   

print(f"🚀 학습 장치: {DEVICE}")

# ==========================================
# 모델 및 학습 준비
# ==========================================
model = CrossAttentionFusion(num_users=NUM_USERS).to(DEVICE)

criterion_cls = nn.CrossEntropyLoss()
criterion_align = AlignmentLoss()
criterion_spread = SpreadControlLoss()

optimizer = optim.Adam(model.parameters(), lr=LR)

# ==========================================
# 학습 실행
# ==========================================
model.train()

for epoch in range(EPOCHS):
    total_loss = 0
    correct = 0
    total_samples = 0
    
    # DataLoader를 tqdm으로 감싸서 진행률 바 생성
    # desc: 바 왼쪽에 표시될 설명 (Epoch 번호)
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=True)
    
    for batch_idx, ((ppg, temp), labels) in enumerate(progress_bar):
        # 데이터 GPU로 이동
        ppg = ppg.to(DEVICE)
        temp = temp.to(DEVICE)
        labels = labels.to(DEVICE)
        
        optimizer.zero_grad()
        
        # 1. Forward
        outputs, z_p, z_t = model(ppg, temp)
        
        # 2. Loss 계산
        loss_cls = criterion_cls(outputs, labels)
        
        # 정렬 손실 (Alignment: 양방향)
        loss_align = criterion_align(z_p, z_t) + criterion_align(z_t, z_p)
        
        # 분산 제어 손실 (Spread)
        loss_spread = criterion_spread(z_p) + criterion_spread(z_t)
        
        # 최종 Loss 합산
        loss = loss_cls + (LAMBDA_A * loss_align) + (LAMBDA_S * loss_spread)
        
        # 3. Backward & Update
        loss.backward()
        optimizer.step()
        
        # 통계 업데이트
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # 실시간 정확도 계산
        current_acc = 100 * correct / total_samples
        
        # TQDM 바 우측에 실시간 정보 표시 (Loss, Accuracy)
        progress_bar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{current_acc:.2f}%",
            'Cls': f"{loss_cls.item():.4f}" # 분류 Loss만 따로 보고 싶다면 추가
        })

    # 에폭 종료 후 평균 기록 출력
    avg_loss = total_loss / len(dataloader)
    final_acc = 100 * correct / total_samples
    print(f"✨ Epoch {epoch+1} Summary - Avg Loss: {avg_loss:.4f}, Accuracy: {final_acc:.2f}%")

print("🏁 모든 학습 완료!")

🚀 학습 장치: cuda


Epoch 1/15:   0%|          | 0/30967 [00:00<?, ?it/s]

Epoch 1/15: 100%|██████████| 30967/30967 [16:25<00:00, 31.44it/s, Loss=2.0711, Acc=20.13%, Cls=2.0473]


✨ Epoch 1 Summary - Avg Loss: 2.6212, Accuracy: 20.13%


Epoch 2/15: 100%|██████████| 30967/30967 [16:26<00:00, 31.38it/s, Loss=2.7368, Acc=22.34%, Cls=2.7006]


...

✨ Epoch 13 Summary - Avg Loss: 2.2851, Accuracy: 25.52%


Epoch 14/15: 100%|██████████| 30967/30967 [16:19<00:00, 31.62it/s, Loss=2.6416, Acc=25.62%, Cls=2.6389]


✨ Epoch 14 Summary - Avg Loss: 2.2807, Accuracy: 25.62%


Epoch 15/15: 100%|██████████| 30967/30967 [16:02<00:00, 32.16it/s, Loss=2.0432, Acc=25.72%, Cls=2.0405]

✨ Epoch 15 Summary - Avg Loss: 2.2774, Accuracy: 25.72%
🏁 모든 학습 완료!
